In [ ]:
import tkinter as tk
from tkinter import filedialog, ttk
from tkinter import *
import cv2
import numpy as np

# Define `net` as a global variable (if necessary)
global net

# Load YOLO model and COCO class labels
model_path = "yolov3.weights"
config_path = "yolov3.cfg"

# Load the YOLO model with GPU acceleration
net = cv2.dnn.readNet(model_path, config_path)

def Label_objects(net):
    layer_names = net.getLayerNames()
    unconnected_out_layers = net.getUnconnectedOutLayers()
    if isinstance(unconnected_out_layers, int):
        return [layer_names[unconnected_out_layers - 1]]  # Return a list with a single layer
    else:
        return [layer_names[i - 1] for i in unconnected_out_layers]  # Multiple layers

def detect_objects(filename, output_window):
    """Detects objects in a video using the loaded YOLO model."""

    # Check if the YOLO model has been loaded
    if net is None:
        print("Please select a video file first.")
        return

    # Load the COCO class names
    classes = []
    with open("coco.names", "r") as f:
        classes = [line.strip() for line in f.readlines()]

    # Load the video
    cap = cv2.VideoCapture(filename)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    target_width, target_height = 600, 440  # Set the target size for resizing

    while True:
        ret, image = cap.read()
        if not ret:
            break

        # Resize the frame to the target size
        image = cv2.resize(image, (target_width, target_height))

        # Create a blob from the resized image
        blob = cv2.dnn.blobFromImage(image, 1/255, (416, 416), swapRB=True, crop=False)

        # Set the blob as the input to the network
        net.setInput(blob)

        # Perform a forward pass to obtain the output layer values
        outs = net.forward(Label_objects(net))

        # Process the outputs and perform non-maximum suppression
        post_process(image, outs, classes)

        # Display the resized image with the bounding boxes and labels
        cv2.imshow(output_window, image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

def post_process(image, outs, classes):
    frame_height, frame_width = image.shape[:2]
    conf_threshold = 0.5
    nms_threshold = 0.4

    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > conf_threshold:
                center_x = int(detection[0] * frame_width)
                center_y = int(detection[1] * frame_height)
                w = int(detection[2] * frame_width)
                h = int(detection[3] * frame_height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])

    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

    if len(indices) > 0:  # Check if indices is not empty
        for i in indices.flatten():
            box = boxes[i]
            x, y, w, h = box[0], box[1], box[2], box[3]
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(image, classes[class_ids[i]], (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

def browse_video():
    """Loads the YOLO model, prompts for a video file, and launches object detection."""
    
    if net is None:
        print("Error: Could not load the model. Please check the file paths.")
        return

    # Open a dialog to select a video file
    filename = filedialog.askopenfilename(filetypes=[("Video files", "*.mp4;*.avi")])
    if filename:
        detect_objects(filename, "Object Detection")

def start_webcam_streaming():
    """Starts webcam object detection."""

    if net is None:
        print("Error: Could not load the model. Please check the file paths.")
        return

    detect_objects(0, "Webcam Object Detection")

# Create Tkinter GUI

from PIL import Image,ImageTk

root = tk.Tk()
root.title("Object Detection")

frame = tk.Frame(root, width=1920, height=1080)

msg=Message(frame,text="Object Tracking using Deep Learning")

# Add a background image to the frame
bg_image = Image.open("Image.png")
bg_image = bg_image.resize((1580, 850), Image.Resampling.LANCZOS)
bg_image = ImageTk.PhotoImage(bg_image)
background_label = tk.Label(frame, image=bg_image)
background_label.place(x=0, y=0, relwidth=1, relheight=1)



browse_button = ttk.Button(root, text="Browse System Video", command=browse_video).place(x=700, y=400)
webcam_button = ttk.Button(root, text="Start Webcam Streaming", command=start_webcam_streaming).place(x=690, y=450)

frame.propagate(0)

frame.pack()
root.mainloop()


In [ ]:
#pip install Pillow